In [134]:
import pandas as pd 
import numpy as np
import matplotlib
import lightgbm as lgb
from sklearn.metrics import accuracy_score,recall_score,f1_score
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

# contest_basic_train表的数据分析及特征工程

In [135]:
train=pd.read_csv('data/contest_basic_train.tsv', sep='\t', header=0)

In [136]:
train.head()

,REPORT_ID,ID_CARD,LOAN_DATE,AGENT,IS_LOCAL,WORK_PROVINCE,EDU_LEVEL,MARRY_STATUS,SALARY,HAS_FUND,Y
0,8787,320382**********13,2017/2/15,APP,本地籍,320000.0,本科,已婚,3.0,0.0,0
1,9410,120223**********8X,2017/3/13,wechat,本地籍,120000.0,NaN,已婚,6.0,1.0,0
2,22999,330226**********90,2017/2/27,APP,本地籍,NaN,专科及以下,已婚,4.0,0.0,0
3,24820,230105**********20,2017/2/24,APP,本地籍,230000.0,专科及以下,已婚,2.0,0.0,1
4,25708,340222**********29,2017/1/16,NaN,本地籍,NaN,专科,未婚,NaN,1.0,0


In [137]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 11 columns):
REPORT_ID        30000 non-null int64
ID_CARD          30000 non-null object
LOAN_DATE        30000 non-null object
AGENT            8952 non-null object
IS_LOCAL         30000 non-null object
WORK_PROVINCE    27742 non-null float64
EDU_LEVEL        26942 non-null object
MARRY_STATUS     30000 non-null object
SALARY           8864 non-null float64
HAS_FUND         29998 non-null float64
Y                30000 non-null int64
dtypes: float64(3), int64(2), object(6)
memory usage: 2.5+ MB


In [138]:
train['EDU_LEVEL'].value_counts(dropna=False)# 缺失值按专科补

专科       7997
本科       6536
高中       6062
专科及以下    4452
NaN      3058
初中       1355
其他        301
硕士研究生     153
博士研究生      51
硕士及以上      35
Name: EDU_LEVEL, dtype: int64

In [139]:
train['EDU_LEVEL']=train['EDU_LEVEL'].fillna('专科')

In [140]:
train['HAS_FUND'].value_counts(dropna=False)

 0.0    16338
 1.0    13660
NaN         2
Name: HAS_FUND, dtype: int64

In [141]:
train['IS_LOCAL'].value_counts(dropna=False)

本地籍     17506
非本地籍    12494
Name: IS_LOCAL, dtype: int64

In [142]:
train['AGENT'].value_counts(dropna=False)#缺失值太多，删除

NaN           21048
APP            6644
wechat         1339
chinapnr        332
rongzhijia      299
huifusdb        240
fenqile          38
ifpp             30
kuaiqian         17
bestpay           8
orgloan           3
weijinhui         2
Name: AGENT, dtype: int64

In [143]:
train=train.drop(['AGENT'],axis=1)

In [144]:
train['MARRY_STATUS'].value_counts(dropna=False)

已婚    19199
未婚     9566
离异      719
离婚      314
其他      161
丧偶       41
Name: MARRY_STATUS, dtype: int64

In [145]:
train['SALARY'].value_counts(dropna=False)#缺失值太多，删除

NaN     21136
 3.0     3057
 4.0     2117
 2.0     1766
 5.0      753
 7.0      599
 6.0      475
 1.0       97
Name: SALARY, dtype: int64

In [146]:
train=train.drop(['SALARY'],axis=1)

In [147]:
train['Y'].value_counts(dropna=False)

0    28125
1     1875
Name: Y, dtype: int64

In [148]:
# One-hot
dumm_edu_level= pd.get_dummies(train['EDU_LEVEL'], prefix='EDU_LEVEL')
final_train = train.join(dumm_edu_level)
final_train=final_train.drop(['EDU_LEVEL'],axis=1)
dumm_marry= pd.get_dummies(train['MARRY_STATUS'], prefix='MARRY_STATUS')
final_train = final_train.join(dumm_marry)
final_train=final_train.drop(['MARRY_STATUS'],axis=1)
dumm_is_local = pd.get_dummies(train['IS_LOCAL'], prefix='IS_LOCAL')
final_train = final_train.join(dumm_is_local)
final_train=final_train.drop(['IS_LOCAL'],axis=1)
dumm_has_fund = pd.get_dummies(train['HAS_FUND'], prefix='HAS_FUND')
final_train = final_train.join(dumm_has_fund)
final_train=final_train.drop(['HAS_FUND'],axis=1)

## contest_ext_crd_cd_ln（机构版征信-贷款）表特征工程

In [154]:
contest_ext_crd_cd_ln=pd.read_csv('data/contest_ext_crd_cd_ln.tsv', sep='\t', header=0)

In [155]:
contest_ext_crd_cd_ln.head()

,loan_id,report_id,state,finance_org,type_dw,currency,guarantee_type,payment_rating,payment_cyc,class5_state,...,balance,remain_payment_cyc,scheduled_payment_amount,actual_payment_amount,curr_overdue_cyc,curr_overdue_amount,recent_pay_date,scheduled_payment_date,open_date,end_date
0,93540,25708,正常,XL,个人消费贷款,人民币,信用/免担保,按月归还,12.0,正常,...,1000.0,9.0,1030.0,1030.0,0.0,0.0,2016-08-11,2016-08-12,2016-05-07,2017-05-07
1,93539,25708,结清,FZ,个人消费贷款,人民币,信用/免担保,按月归还,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-06-14,NaN
2,795426,170491,正常,ORG-WM,个人消费贷款,人民币,信用/免担保,按月归还,12.0,正常,...,5000.0,12.0,0.0,0.0,0.0,0.0,2016-12-12,2016-12-12,2016-12-12,2017-12-12
3,817296,174368,正常,OF,个人消费贷款,人民币,信用/免担保,按月归还,6.0,正常,...,675.0,4.0,178.0,178.0,0.0,0.0,2016-12-03,2016-12-03,2016-10-11,2017-04-03
4,817298,174368,正常,GB,个人消费贷款,人民币,信用/免担保,按月归还,12.0,正常,...,1100.0,11.0,105.0,105.0,0.0,0.0,2016-12-05,2016-12-05,2016-11-17,2017-11-05


In [156]:
contest_ext_crd_cd_ln=contest_ext_crd_cd_ln.fillna(0)

In [157]:
curr_overdue_cyc=contest_ext_crd_cd_ln.groupby(['report_id'],as_index=False)[['curr_overdue_cyc']].sum()
curr_overdue_amount=contest_ext_crd_cd_ln.groupby(['report_id'],as_index=False)[['curr_overdue_amount']].sum()

In [158]:
final_train = pd.merge(final_train, curr_overdue_cyc, left_on=['REPORT_ID'],right_on=['report_id'], how='left')
final_train = pd.merge(final_train, curr_overdue_amount, left_on=['REPORT_ID'],right_on=['report_id'], how='left')
final_train=final_train.drop(['report_id_x'],axis=1)
final_train=final_train.drop(['report_id_y'],axis=1)

In [159]:
final_train.columns

Index(['REPORT_ID', 'ID_CARD', 'LOAN_DATE', 'WORK_PROVINCE', 'Y',
       'EDU_LEVEL_专科', 'EDU_LEVEL_专科及以下', 'EDU_LEVEL_其他', 'EDU_LEVEL_初中',
       'EDU_LEVEL_博士研究生', 'EDU_LEVEL_本科', 'EDU_LEVEL_硕士及以上', 'EDU_LEVEL_硕士研究生',
       'EDU_LEVEL_高中', 'MARRY_STATUS_丧偶', 'MARRY_STATUS_其他', 'MARRY_STATUS_已婚',
       'MARRY_STATUS_未婚', 'MARRY_STATUS_离婚', 'MARRY_STATUS_离异', 'IS_LOCAL_本地籍',
       'IS_LOCAL_非本地籍', 'HAS_FUND_0.0', 'HAS_FUND_1.0', 'curr_overdue_cyc',
       'curr_overdue_amount'],
      dtype='object')

## contest_ext_crd_cd_lnd（机构版征信-贷记卡）表特征

In [160]:
contest_ext_crd_cd_lnd=pd.read_csv('data/contest_ext_crd_cd_lnd.tsv', sep='\t', header=0)

In [161]:
contest_ext_crd_cd_lnd.head()

,report_id,loancard_id,state,finance_org,currency,open_date,credit_limit_amount,guarantee_type,share_credit_limit_amount,used_credit_limit_amount,latest6_month_used_avg_amount,used_highest_amount,scheduled_payment_date,scheduled_payment_amount,actual_payment_amount,recent_pay_date,curr_overdue_cyc,curr_overdue_amount,payment_state,cardtype
0,50598.0,387641,正常,MI,人民币,2014-09-19,13375,信用/免担保,13375,8483.0,8494.0,11351.0,2016-10-05,424.0,8500.0,2016-09-23,0.0,0.0,NNNNNNNNNNN12NNNNNNNNNNN,贷记卡
1,50598.0,387634,正常,MI,美元,2014-09-19,10023,信用/免担保,0,0.0,0.0,427.0,2016-10-05,0.0,0.0,2015-08-27,0.0,0.0,********NNN*************,贷记卡
2,50598.0,387635,正常,OE,人民币,2014-07-09,5000,信用/免担保,5000,4962.0,5281.0,6236.0,2016-10-10,589.0,5080.0,2016-09-25,0.0,0.0,NNNNNNNNNNN1NNNNNNNNNNNN,贷记卡
3,50598.0,387643,正常,LL,人民币,2014-10-27,10000,信用/免担保,10000,9996.0,10251.0,11713.0,2016-11-01,0.0,0.0,2016-09-07,0.0,0.0,NNNNNNNNNNNNNNNNNNNNNNNN,贷记卡
4,50598.0,387636,销户,SM,美元,2008-04-04,6000,信用/免担保,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,贷记卡


In [162]:
contest_ext_crd_cd_lnd=contest_ext_crd_cd_lnd.fillna(0)

In [163]:
curr_overdue_cyc=contest_ext_crd_cd_lnd.groupby(['report_id'],as_index=False)[['curr_overdue_cyc']].sum()
curr_overdue_amount=contest_ext_crd_cd_lnd.groupby(['report_id'],as_index=False)[['curr_overdue_amount']].sum()
curr_overdue_cyc=curr_overdue_cyc.rename(columns={'curr_overdue_cyc':'curr_overdue_cyc_cd_lnd'})
curr_overdue_amount=curr_overdue_amount.rename(columns={'curr_overdue_amount':'curr_overdue_amount_cd_lnd'})

In [164]:
final_train = pd.merge(final_train, curr_overdue_cyc, left_on=['REPORT_ID'],right_on=['report_id'], how='left')
final_train = pd.merge(final_train, curr_overdue_amount, left_on=['REPORT_ID'],right_on=['report_id'], how='left')
final_train=final_train.drop(['report_id_x'],axis=1)
final_train=final_train.drop(['report_id_y'],axis=1)

In [165]:
final_train.columns

Index(['REPORT_ID', 'ID_CARD', 'LOAN_DATE', 'WORK_PROVINCE', 'Y',
       'EDU_LEVEL_专科', 'EDU_LEVEL_专科及以下', 'EDU_LEVEL_其他', 'EDU_LEVEL_初中',
       'EDU_LEVEL_博士研究生', 'EDU_LEVEL_本科', 'EDU_LEVEL_硕士及以上', 'EDU_LEVEL_硕士研究生',
       'EDU_LEVEL_高中', 'MARRY_STATUS_丧偶', 'MARRY_STATUS_其他', 'MARRY_STATUS_已婚',
       'MARRY_STATUS_未婚', 'MARRY_STATUS_离婚', 'MARRY_STATUS_离异', 'IS_LOCAL_本地籍',
       'IS_LOCAL_非本地籍', 'HAS_FUND_0.0', 'HAS_FUND_1.0', 'curr_overdue_cyc',
       'curr_overdue_amount', 'curr_overdue_cyc_cd_lnd',
       'curr_overdue_amount_cd_lnd'],
      dtype='object')

## contest_ext_crd_is_ovdsummary（机构版征信-逾期(透支)信息汇总）

In [166]:
contest_ext_crd_is_ovdsummary=pd.read_csv('data/contest_ext_crd_is_ovdsummary.csv')

In [167]:
contest_ext_crd_is_ovdsummary.head()

,REPORT_ID,TYPE_DW,COUNT_DW,MONTHS,HIGHEST_OA_PER_MON,MAX_DURATION
0,9410,贷款逾期,0,0,0.0,0
1,9410,贷记卡逾期,1,1,0.0,1
2,9410,准贷记卡60天以上透支,0,0,0.0,0
3,24820,贷款逾期,0,0,0.0,0
4,24820,贷记卡逾期,3,6,440.0,1


In [168]:
contest_ext_crd_is_ovdsummary.shape

(76212, 6)

In [169]:
# 贷款逾期特征
contest_ext_crd_is_ovdsummary_daikuan=contest_ext_crd_is_ovdsummary[contest_ext_crd_is_ovdsummary['TYPE_DW']=='贷款逾期']
contest_ext_crd_is_ovdsummary_daikuan=contest_ext_crd_is_ovdsummary_daikuan.rename(columns={
    'COUNT_DW':'COUNT_DW_daikuan',
    'MONTHS':'MONTHS_daikuan',
    'HIGHEST_OA_PER_MON':'HIGHEST_OA_PER_MON_daikuan',
    'MAX_DURATION':'MAX_DURATION_daikuan'
})
contest_ext_crd_is_ovdsummary_daikuan=contest_ext_crd_is_ovdsummary_daikuan.drop(['TYPE_DW'],axis=1)
final_train = pd.merge(final_train, contest_ext_crd_is_ovdsummary_daikuan, on=['REPORT_ID'], how='left')

In [170]:
# 准贷记卡60天以上透支特征
contest_ext_crd_is_ovdsummary_zhundaijika=contest_ext_crd_is_ovdsummary[contest_ext_crd_is_ovdsummary['TYPE_DW']=='准贷记卡60天以上透支']
contest_ext_crd_is_ovdsummary_zhundaijika=contest_ext_crd_is_ovdsummary_zhundaijika.rename(columns={
    'COUNT_DW':'COUNT_DW_zhundaijika',
    'MONTHS':'MONTHS_zhundaijika',
    'HIGHEST_OA_PER_MON':'HIGHEST_OA_PER_MON_zhundaijika',
    'MAX_DURATION':'MAX_DURATION_zhundaijika'
})
contest_ext_crd_is_ovdsummary_zhundaijika=contest_ext_crd_is_ovdsummary_zhundaijika.drop(['TYPE_DW'],axis=1)
final_train = pd.merge(final_train, contest_ext_crd_is_ovdsummary_zhundaijika, on=['REPORT_ID'], how='left')

In [171]:
# 贷记卡逾期特征
contest_ext_crd_is_ovdsummary_daijika=contest_ext_crd_is_ovdsummary[contest_ext_crd_is_ovdsummary['TYPE_DW']=='贷记卡逾期']
contest_ext_crd_is_ovdsummary_daijika=contest_ext_crd_is_ovdsummary_daijika.rename(columns={
    'COUNT_DW':'COUNT_DW_daijika',
    'MONTHS':'MONTHS_daijika',
    'HIGHEST_OA_PER_MON':'HIGHEST_OA_PER_MON_daijika',
    'MAX_DURATION':'MAX_DURATION_daijika'
})
contest_ext_crd_is_ovdsummary_daijika=contest_ext_crd_is_ovdsummary_daijika.drop(['TYPE_DW'],axis=1)
final_train = pd.merge(final_train, contest_ext_crd_is_ovdsummary_daijika, on=['REPORT_ID'], how='left')

In [172]:
final_train.columns

Index(['REPORT_ID', 'ID_CARD', 'LOAN_DATE', 'WORK_PROVINCE', 'Y',
       'EDU_LEVEL_专科', 'EDU_LEVEL_专科及以下', 'EDU_LEVEL_其他', 'EDU_LEVEL_初中',
       'EDU_LEVEL_博士研究生', 'EDU_LEVEL_本科', 'EDU_LEVEL_硕士及以上', 'EDU_LEVEL_硕士研究生',
       'EDU_LEVEL_高中', 'MARRY_STATUS_丧偶', 'MARRY_STATUS_其他', 'MARRY_STATUS_已婚',
       'MARRY_STATUS_未婚', 'MARRY_STATUS_离婚', 'MARRY_STATUS_离异', 'IS_LOCAL_本地籍',
       'IS_LOCAL_非本地籍', 'HAS_FUND_0.0', 'HAS_FUND_1.0', 'curr_overdue_cyc',
       'curr_overdue_amount', 'curr_overdue_cyc_cd_lnd',
       'curr_overdue_amount_cd_lnd', 'COUNT_DW_daikuan', 'MONTHS_daikuan',
       'HIGHEST_OA_PER_MON_daikuan', 'MAX_DURATION_daikuan',
       'COUNT_DW_zhundaijika', 'MONTHS_zhundaijika',
       'HIGHEST_OA_PER_MON_zhundaijika', 'MAX_DURATION_zhundaijika',
       'COUNT_DW_daijika', 'MONTHS_daijika', 'HIGHEST_OA_PER_MON_daijika',
       'MAX_DURATION_daijika'],
      dtype='object')

## contest_ext_crd_cd_lnd_ovd表特征工程

In [173]:
contest_ext_crd_cd_lnd_ovd=pd.read_csv('data/contest_ext_crd_cd_lnd_ovd.csv')

In [174]:
contest_ext_crd_cd_lnd_ovd.head()

,REPORT_ID,MONTH_DW,LAST_MONTHS,AMOUNT
0,24820,2014.02,1.0,440.0
1,24820,2013.07,1.0,95.0
2,54038,2014.06,1.0,10.0
3,54038,--,NaN,NaN
4,88203,2014.02,1.0,1321.0


In [175]:
contest_ext_crd_cd_lnd_ovd=contest_ext_crd_cd_lnd_ovd.fillna(0)

In [176]:
LAST_MONTHS=contest_ext_crd_cd_lnd_ovd.groupby(['REPORT_ID'],as_index=False)[['LAST_MONTHS']].sum()
AMOUNT=contest_ext_crd_cd_lnd_ovd.groupby(['REPORT_ID'],as_index=False)[['AMOUNT']].sum()
final_train = pd.merge(final_train, LAST_MONTHS, on=['REPORT_ID'], how='left')
final_train = pd.merge(final_train, AMOUNT, on=['REPORT_ID'], how='left')

## 训练数据集、测试数据集划分、模型训练

In [181]:
final_train = final_train.fillna(0)
final_train=final_train.drop(['REPORT_ID','ID_CARD','LOAN_DATE','WORK_PROVINCE'],axis=1)
final_train=final_train.sample(frac=1,random_state=13) 
train_y,test_y = final_train['Y'].values[:24000],final_train['Y'].values[24000:]
final_train=final_train.drop(['Y'],axis=1)
train_x,test_x = final_train[:24000].values,final_train[24000:].values

In [182]:
model = LinearSVC(C=3)
model.fit(train_x, train_y)
y_pre=model.predict(test_x)
print('-'*15+' TRAIN '+'-'*15)
y_train_pre=model.predict(train_x)
print('accuracy_score: ',accuracy_score(train_y,y_train_pre))
print('recall_score: ',recall_score(train_y,y_train_pre))
print('f1_score: ',f1_score(train_y,y_train_pre))
print('-'*15+' TEST '+'-'*15)
print('accuracy_score: ',accuracy_score(test_y,y_pre))
print('recall_score: ',recall_score(test_y,y_pre))
print('f1_score: ',f1_score(test_y,y_pre))

--------------- TRAIN ---------------
accuracy_score:  0.879083333333
recall_score:  0.0702099737533
f1_score:  0.0686777920411
--------------- TEST ---------------
accuracy_score:  0.8845
recall_score:  0.0626780626781
f1_score:  0.0597014925373
